In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from scipy import stats 

In [3]:
#load the dataset
amx= pd.read_parquet('train_data.parquet')

In [4]:
#create random sampling
amx_1=amx.sample(n=500000)
t1=amx.sample(n=100000)
t2=amx.sample(n=100000)
t3=amx.sample(n=100000)
t4=amx.sample(n=100000)
t5=amx.sample(n=100000)

In [5]:
#check shape of data
amx_1.shape,t1.shape,t2.shape,t3.shape,t4.shape,t5.shape

((500000, 191),
 (100000, 191),
 (100000, 191),
 (100000, 191),
 (100000, 191),
 (100000, 191))

In [28]:
amx_samp=pd.concat([amx_1,t1,t2,t3,t4,t5],axis=0)

2          O
3          O
15         O
31         R
31         R
          ..
5531430    U
5531432    U
5531436    R
5531439    U
5531442    O
Name: D_64, Length: 1000000, dtype: object

In [32]:
amx_samp.shape
amx_samp.head()
sort=amx_samp.sort_values(["customer_ID","S_2"])
sort["D_64"]=sort["D_64"].ffill().bfill()
cat=pd.concat([sort.loc[sort["target"]==0].groupby("customer_ID")[["D_63","D_64"]].max(),sort.loc[sort["target"]==1].groupby("customer_ID")[["D_63","D_64"]].max()],axis=0,ignore_index=True)


In [33]:
#sort=amx_samp.sort_values(["customer_ID","S_2"])
#sort["D_64"]=sort["D_64"].ffill().bfill()

#sort1=pd.concat([sort,pd.get_dummies(sort[["D_63","D_64"]],drop_first=True)],axis=1)
num=pd.concat([sort.loc[sort["target"]==0].groupby("customer_ID").median(),sort.loc[sort["target"]==1].groupby("customer_ID").median()],axis=0,ignore_index=True)

num.shape,cat.shape


((399877, 187), (399877, 2))

In [49]:
df=pd.get_dummies(pd.concat([num,cat],axis=1),drop_first=True)
df=df.ffill().bfill()
df.shape

(399877, 195)

In [102]:
import numpy as np
corr_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.75
to_drop = [column for column in upper.columns if any(upper[column] > 0.75)]

# Drop features 
#df.drop(to_drop, axis=1, inplace=True)

In [80]:
upper

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,...,D_145,target,D_63_CO,D_63_CR,D_63_XL,D_63_XM,D_63_XZ,D_64_O,D_64_R,D_64_U
P_2,NaN,0.18885,0.367148,0.557105,0.475991,0.326299,0.310715,0.448749,0.235435,0.287163,...,0.166109,0.614998,0.041110,0.053109,0.005631,0.002169,0.020095,0.254141,0.087495,0.202303
D_39,NaN,NaN,0.188734,0.195538,0.202108,0.066271,0.444340,0.182154,0.066257,0.049564,...,0.023412,0.186362,0.188220,0.091948,0.018958,0.018609,0.006330,0.014754,0.004588,0.013072
B_1,NaN,NaN,NaN,0.647279,0.236801,0.163704,0.182016,0.738076,0.096518,0.083217,...,0.078278,0.393132,0.103366,0.121281,0.002188,0.018355,0.005767,0.070520,0.025582,0.055792
B_2,NaN,NaN,NaN,NaN,0.320103,0.239378,0.255436,0.728015,0.150239,0.151505,...,0.127359,0.497168,0.135589,0.171234,0.001084,0.021994,0.011971,0.128990,0.041340,0.106674
R_1,NaN,NaN,NaN,NaN,NaN,0.222936,0.277583,0.279042,0.143223,0.189745,...,0.076915,0.386730,0.019015,0.043066,0.006733,0.001787,0.007988,0.082478,0.029465,0.065765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
D_63_XM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.004595,0.001991,0.001655,0.001186
D_63_XZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.006870,0.002901,0.004845
D_64_O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.433248,0.717138
D_64_R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.303831


In [105]:
for cols in upper.drop(["D_63_CR","D_63_CO","D_63_XL","D_63_XM","D_63_XZ","D_64_O","D_64_R","D_64_U"],axis=1).columns:
    for i in upper.loc[(upper[cols]>0.75) ].index:
        print(cols,i)

B_11 B_1
R_4 R_2
S_7 S_3
D_55 D_48
D_61 D_48
D_61 D_55
B_15 B_14
B_16 B_2
B_18 B_2
B_18 B_16
B_19 B_3
B_20 B_2
B_20 B_3
B_20 B_16
B_20 B_18
B_22 B_3
S_15 S_8
B_23 B_7
D_74 D_58
D_75 D_44
D_75 D_58
D_75 D_74
D_77 D_62
R_8 R_5
B_30 B_19
B_30 B_22
R_21 S_20
B_33 B_2
B_33 B_16
B_33 B_18
B_33 B_20
S_24 S_22
D_104 D_103
D_107 D_103
D_107 D_104
B_37 B_1
B_37 B_11
B_38 B_16
D_118 D_115
D_119 D_115
D_119 D_118
D_129 D_128
D_131 D_79
D_141 D_139
D_143 D_139
D_143 D_141


In [120]:
df1["S1_new"]=(df1["S_7"]+df1["S_3"])/2
df1["S2_new"]=(df1["S_15"]+df1["S_8"])/2
df1["S3_new"]=(df1["S_22"]+df1["S_24"])/2

In [123]:
s_drop=["S_7","S_3","S_8","S_15","S_22","S_24"]
df1.drop(s_drop,axis=1,inplace=True)

In [116]:
df1["R1_new"]=(df1["R_4"]+df1["R_2"])/2
df1["R2_new"]=(df1["R_5"]+df1["R_8"])/2
df1["R3_new"]=(df1["R_21"]+df1["S_20"])/2

In [119]:
r_drop=["R_2","R_4","R_5","R_8","R_21","S_20"]
df1.drop(r_drop,axis=1,inplace=True)

In [112]:
df1["D1_new"]=(df1["D_55"]+df1["D_48"]+df1["D_61"])/3
df1["D2_new"]=(df1["D_74"]+df1["D_58"]+df1["D_75"]+df1["D_44"])/4
df1["D3_new"]=(df1["D_77"]+df1["D_62"])/2
df1["D4_new"]=(df1["D_103"]+df1["D_104"]+df1["D_107"])/3
df1["D5_new"]=(df1["D_118"]+df1["D_119"]+df1["D_115"])/3
df1["D6_new"]=(df1["D_129"]+df1["D_128"])/2
df1["D7_new"]=(df1["D_131"]+df1["D_79"])/2
df1["D8_new"]=(df1["D_141"]+df1["D_143"]+df1["D_139"])/3

In [113]:
df1.columns

Index(['P_2', 'D_39', 'R_1', 'S_3', 'D_41', 'D_42', 'D_43', 'D_44', 'B_4',
       'D_45',
       ...
       'B3_New', 'B4_New', 'D1_new', 'D2_new', 'D3_new', 'D4_new', 'D5_new',
       'D6_new', 'D7_new', 'D8_new'],
      dtype='object', length=190)

In [115]:
d_drop=["D_55","D_48","D_61","D_74","D_58","D_75",
        "D_44","D_77","D_62","D_103","D_104","D_107","D_118","D_119","D_115","D_128","D_129","D_131","D_79",
       "D_141","D_143","D_139"]
df1.drop(d_drop,axis=1,inplace=True)

In [107]:
df1=df.copy()
df1["B1_New"]=(df1["B_11"] + df1["B_1"]+df1["B_37"])/3
df1["B2_New"]=(df1["B_15"] + df1["B_14"])/2
df1["B3_New"]=(df1["B_16"] + df1["B_2"]+df["B_18"]+df1["B_20"]+df1["B_33"]+df1["B_38"]+df1["B_3"]+df1["B_19"]+df1["B_22"]+df1["B_30"])/10
df1["B4_New"]=(df1["B_23"] + df1["B_7"])/2


In [110]:
b_drop=["B_11", "B_1","B_15", "B_14","B_16", "B_2","B_18","B_19", "B_3","B_20","B_22","B_23",
        "B_7","B_30","B_33","B_37","B_11","B_38" ]
df1.drop(b_drop,axis=1,inplace=True)

Index(['P_2', 'D_39', 'R_1', 'S_3', 'D_41', 'D_42', 'D_43', 'D_44', 'B_4',
       'D_45',
       ...
       'D_63_XL', 'D_63_XM', 'D_63_XZ', 'D_64_O', 'D_64_R', 'D_64_U', 'B1_New',
       'B2_New', 'B3_New', 'B4_New'],
      dtype='object', length=182)

In [128]:
corr_matrix = df1.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.75
#to_drop = [column for column in upper.columns if any(upper[column] > 0.75)]

# Drop features 
#df.drop(to_drop, axis=1, inplace=True)
upper["target"]

P_2       0.614998
D_39      0.186362
R_1       0.386730
D_41      0.245228
D_42      0.311544
            ...   
R2_new         NaN
R3_new         NaN
S1_new         NaN
S2_new         NaN
S3_new         NaN
Name: target, Length: 162, dtype: float64

In [136]:
for i in df1.columns:
    if df1[i].var()<0.005

0.058415573090314865
0.05275736004114151
0.03997426480054855
0.034856297075748444
0.04598183184862137
0.0360322967171669
0.04725843295454979
0.058502670377492905
0.12681220471858978
0.021729016676545143
0.05527974292635918
0.10651830583810806
0.6124359369277954
0.24496319890022278
0.2763262987136841
0.05497455596923828
0.07379838824272156
0.041924651712179184
0.029831279069185257
0.02322932705283165
19.63359832763672
0.025889422744512558
0.1231040209531784
0.17394214868545532
0.01695474050939083
0.693972110748291
0.04415788874030113
0.22519683837890625
0.037066079676151276
0.0357850082218647
0.13189586997032166
0.03147759288549423
0.19532091915607452
0.15545828640460968
0.022774530574679375
1.8163409233093262
0.026262907311320305
0.27148813009262085
0.08151068538427353
0.4030681252479553
451.8224792480469
0.04639692232012749
0.07895229756832123
0.03761737793684006
0.03677292540669441
0.1235516220331192
0.0749252662062645
0.07629939168691635
1.152436375617981
0.04083798825740814
6.73522

In [149]:
import scipy.stats as stats
insig=["target"]
for i in df1.columns:
        zer=df1.loc[df1["target"]==0,i]
        one=df1.loc[df1["target"]==1,i]
        if stats.ttest_ind(zer,one).pvalue>(10)**(-90):
            insig.append(i)

In [168]:
#df1[insig].corr()["target"]
#df1.corr()[["target"]].loc[abs(df1.corr()["target"])>0.01].sort_values("target")
df1.corr()[["target"]].loc[abs(df1.corr()["target"])>0.01].sort_values("target")

,target
P_2,-0.614998
D3_new,-0.392422
D_47,-0.283030
P_3,-0.282313
R_27,-0.279472
...,...
B4_New,0.429431
D2_new,0.478426
B_9,0.497119
D1_new,0.536556


In [167]:
df1.drop("D_87",axis=1,inplace=True)

In [169]:
df1.shape

(399877, 161)

In [174]:
X=df1.drop("target",axis=1)
y=df1["target"]

In [182]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(X,y,test_size=0.25,random_state=10,stratify=y)

In [183]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
model_rf=rf.fit(xtrain,ytrain)
pred_rf=model_rf.predict(xtest)

from sklearn.metrics import f1_score,classification_report, accuracy_score
#print("accuracy score",accuracy_score(ytest,pred))
print("f1 score1",f1_score(ytest,pred_rf))

#print("classification report",classification_report(ytest,pred))

f1 score1 0.9076870625513908


In [189]:
print("classification_report",classification_report(ytest,pred_rf))

classification_report               precision    recall  f1-score   support

         0.0       0.96      0.98      0.97     74591
         1.0       0.92      0.89      0.91     25379

    accuracy                           0.95     99970
   macro avg       0.94      0.93      0.94     99970
weighted avg       0.95      0.95      0.95     99970



In [188]:
print("accuracy_score",accuracy_score(ytest,pred_rf))

accuracy_score 0.9539561868560568


In [184]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()

In [185]:
from sklearn.model_selection import StratifiedKFold
f1=[]
skf=StratifiedKFold(n_splits=5,random_state=None)
skf.get_n_splits(X,y)
for train_index,test_index in skf.split(X,y):
    print("Train:", train_index, "Validation:", test_index)
    x1_train,x1_test=X.iloc[train_index],X.iloc[test_index]
    y1_train,y1_test=y.iloc[train_index],y.iloc[test_index]
    classifier.fit(x1_train,y1_train)
    prediction=classifier.predict(x1_test)
    score=f1_score(y1_test,prediction)
    f1.append(score)

print(f1)
    

Train: [ 59672  59673  59674 ... 399874 399875 399876] Validation: [     0      1      2 ... 318661 318662 318663]
Train: [     0      1      2 ... 399874 399875 399876] Validation: [ 59672  59673  59674 ... 338965 338966 338967]
Train: [     0      1      2 ... 399874 399875 399876] Validation: [119344 119345 119346 ... 359268 359269 359270]
Train: [     0      1      2 ... 399874 399875 399876] Validation: [179016 179017 179018 ... 379571 379572 379573]
Train: [     0      1      2 ... 379571 379572 379573] Validation: [238688 238689 238690 ... 399874 399875 399876]
[0.8905768218395127, 0.9061594659276988, 0.9029709944403914, 0.88271620625683, 0.9086614568937357]


In [193]:
print(classification_report(y1_test,prediction))

              precision    recall  f1-score   support

         0.0       0.96      0.98      0.97     59672
         1.0       0.93      0.89      0.91     20303

    accuracy                           0.95     79975
   macro avg       0.95      0.93      0.94     79975
weighted avg       0.95      0.95      0.95     79975



In [195]:
from sklearn.linear_model import LogisticRegression
log=LogisticRegression()
from sklearn.model_selection import StratifiedKFold
f1=[]
skf=StratifiedKFold(n_splits=5,random_state=None)
skf.get_n_splits(X,y)
for train_index,test_index in skf.split(X,y):
    print("Train:", train_index, "Validation:", test_index)
    x1_train,x1_test=X.iloc[train_index],X.iloc[test_index]
    y1_train,y1_test=y.iloc[train_index],y.iloc[test_index]
    log.fit(x1_train,y1_train)
    pred_log=log.predict(x1_test)
    score=f1_score(y1_test,pred_log)
    f1.append(score)

print(f1)

Train: [ 59672  59673  59674 ... 399874 399875 399876] Validation: [     0      1      2 ... 318661 318662 318663]
Train: [     0      1      2 ... 399874 399875 399876] Validation: [ 59672  59673  59674 ... 338965 338966 338967]
Train: [     0      1      2 ... 399874 399875 399876] Validation: [119344 119345 119346 ... 359268 359269 359270]
Train: [     0      1      2 ... 399874 399875 399876] Validation: [179016 179017 179018 ... 379571 379572 379573]
Train: [     0      1      2 ... 379571 379572 379573] Validation: [238688 238689 238690 ... 399874 399875 399876]
[0.8681636726546906, 0.872234718342932, 0.8714881366027996, 0.8626497824660332, 0.8853762750376646]


In [190]:
from lightgbm import LGBMClassifier
light=LGBMClassifier()
model_lgb=light.fit(xtrain,ytrain)
pred_lgb=model_lgb.predict(xtest)

from sklearn.metrics import f1_score,classification_report, accuracy_score
#print("accuracy score",accuracy_score(ytest,pred))

print("f1 score1",f1_score(ytest,pred_lgb))

f1 score1 0.9076870625513908


In [191]:
print("classification_report",classification_report(ytest,pred_lgb))

classification_report               precision    recall  f1-score   support

         0.0       0.99      0.98      0.98     74591
         1.0       0.95      0.96      0.96     25379

    accuracy                           0.98     99970
   macro avg       0.97      0.97      0.97     99970
weighted avg       0.98      0.98      0.98     99970



In [198]:
df1.shape

(399877, 161)

In [197]:
c181=['P_2', 'D_39', 'R_1', 'S_3', 'D_41', 'D_42', 'D_43', 'D_44', 'B_4', 'D_45', 'B_5', 'R_2', 'D_46', 'D_47', 'D_48', 'D_49', 'B_6', 'B_8', 'D_50', 'D_51', 'B_9', 'R_3', 'D_52', 'P_3', 'B_10', 'D_53', 'S_5', 'S_6', 'D_54', 'R_4', 'S_7', 'B_12', 'S_8', 'D_55', 'D_56', 'B_13', 'R_5', 'D_58', 'S_9', 'D_59', 'D_60', 'D_61', 'S_11', 'D_62', 'D_65', 'B_17', 'D_66', 'D_68', 'S_12', 'R_6', 'S_13', 'B_21', 'D_69', 'D_70', 'D_71', 'D_72', 'S_15', 'D_73', 'P_4', 'D_74', 'D_75', 'D_76', 'B_24', 'R_7', 'D_77', 'B_25', 'B_26', 'D_78', 'D_79', 'R_8', 'R_9', 'S_16', 'D_80', 'R_10', 'R_11', 'B_27', 'D_81', 'D_82', 'S_17', 'R_12', 'B_28', 'R_13', 'D_83', 'R_14', 'R_15', 'D_84', 'R_16', 'B_29', 'S_18', 'D_86', 'R_17', 'R_18', 'D_88', 'B_31', 'S_19', 'R_19', 'B_32', 'S_20', 'R_20', 'R_21', 'D_89', 'R_22', 'R_23', 'D_91', 'D_92', 'D_93', 'D_94', 'R_24', 'R_25', 'D_96', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'D_102', 'D_103', 'D_104', 'D_105', 'D_106', 'D_107', 'B_36', 'R_26', 'R_27', 'D_108', 'D_109', 'D_110', 'D_111', 'B_39', 'D_112', 'B_40', 'S_27', 'D_113', 'D_114', 'D_115', 'D_116', 'D_117', 'D_118', 'D_119', 'D_120', 'D_121', 'D_122', 'D_123', 'D_124', 'D_125', 'D_126', 'D_127', 'D_128', 'D_129', 'B_41', 'B_42', 'D_130', 'D_131', 'D_132', 'D_133', 'R_28', 'D_134', 'D_135', 'D_136', 'D_137', 'D_138', 'D_139', 'D_140', 'D_141', 'D_142', 'D_143', 'D_144', 'D_145', 'target', 'D_63_CO', 'D_63_CR', 'D_63_XL', 'D_63_XM', 'D_63_XZ', 'D_64_O', 'D_64_R', 'D_64_U', 'B1_New', 'B2_New', 'B3_New', 'B4_New']


In [199]:
for i in c181:
    if i not in df1.columns:
        print(i)

S_3
D_44
R_2
D_48
R_4
S_7
S_8
D_55
R_5
D_58
D_61
D_62
S_15
D_74
D_75
D_77
D_79
R_8
S_20
R_21
S_22
S_24
D_103
D_104
D_107
D_115
D_118
D_119
D_128
D_129
D_131
D_139
D_141
D_143
